## Importing libraries

In [60]:
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

In [61]:
print(tf.reduce_sum(tf.random.normal([1000, 1000])))

tf.Tensor(582.66626, shape=(), dtype=float32)


In [62]:
print(tf.config.list_physical_devices('CPU'))

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]


## Data preprocessing

### Training Image Preprocessing

In [63]:
training_set=tf.keras.utils.image_dataset_from_directory(
    "train/train",
    labels="inferred",
    label_mode="categorical",
    class_names=None,
    color_mode="rgb",
    batch_size=32,
    image_size=(128, 128),
    shuffle=True,
    seed=None,
    validation_split=None,
    subset=None,
    interpolation="bilinear",
    follow_links=False,
    crop_to_aspect_ratio=False,
)

Found 47391 files belonging to 26 classes.


## validation image processing

In [64]:
validation_set= training_set=tf.keras.utils.image_dataset_from_directory(
    "valid/valid",
    labels="inferred",
    label_mode="categorical",
    class_names=None,
    color_mode="rgb",
    batch_size=32,
    image_size=(128, 128),
    shuffle=True,
    seed=None,
    validation_split=None,
    subset=None,
    interpolation="bilinear",
    follow_links=False,
    crop_to_aspect_ratio=False,
)

Found 17572 files belonging to 38 classes.


In [65]:
training_set

<BatchDataset element_spec=(TensorSpec(shape=(None, 128, 128, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 38), dtype=tf.float32, name=None))>

In [66]:
for x,y in training_set:
    print(x,x.shape)
    print(y,y.shape)
    break

tf.Tensor(
[[[[162.5  150.5  160.5 ]
   [172.75 160.75 170.75]
   [155.   143.   153.  ]
   ...
   [163.25 152.25 160.25]
   [160.   149.   157.  ]
   [160.75 149.75 157.75]]

  [[156.   144.   154.  ]
   [157.   145.   155.  ]
   [153.75 141.75 151.75]
   ...
   [165.75 154.75 162.75]
   [171.75 160.75 168.75]
   [158.   147.   155.  ]]

  [[156.75 144.75 154.75]
   [158.   146.   156.  ]
   [154.5  142.5  152.5 ]
   ...
   [160.   149.   157.  ]
   [152.5  141.5  149.5 ]
   [158.25 147.25 155.25]]

  ...

  [[ 57.5   60.5   79.5 ]
   [ 58.5   61.5   80.5 ]
   [ 57.25  60.25  79.25]
   ...
   [159.75 147.75 157.75]
   [166.75 154.75 164.75]
   [169.25 157.25 167.25]]

  [[ 61.5   64.5   83.5 ]
   [ 60.25  63.25  82.25]
   [ 55.5   58.5   77.5 ]
   ...
   [153.75 141.75 151.75]
   [162.5  150.5  160.5 ]
   [161.   149.   159.  ]]

  [[ 60.25  63.25  82.25]
   [ 59.75  62.75  81.75]
   [ 56.    59.    78.  ]
   ...
   [155.5  143.5  153.5 ]
   [166.25 154.25 164.25]
   [182.5  170.5  18

## Building Model

In [67]:
from tensorflow.keras.layers import Dense,Conv2D,MaxPool2D,Flatten,Dropout
from tensorflow.keras.models import Sequential

In [68]:
model= Sequential()

In [69]:
### Building Convulation Layer

In [70]:
model.add(Conv2D(filters=32,kernel_size=3,padding='same',activation='relu', input_shape=[128,128,3]))
model.add(Conv2D(filters=32,kernel_size=3,activation='relu'))
model.add(MaxPool2D(pool_size=2 , strides=2))

In [71]:
model.add(Conv2D(filters=64,kernel_size=3,padding='same',activation='relu', input_shape=[128,128,3]))
model.add(Conv2D(filters=64,kernel_size=3,activation='relu'))
model.add(MaxPool2D(pool_size=2 , strides=2))

In [72]:
model.add(Conv2D(filters=128,kernel_size=3,padding='same',activation='relu', input_shape=[128,128,3]))
model.add(Conv2D(filters=128,kernel_size=3,activation='relu'))
model.add(MaxPool2D(pool_size=2 , strides=2))

In [73]:
model.add(Conv2D(filters=256,kernel_size=3,padding='same',activation='relu'))
model.add(Conv2D(filters=256,kernel_size=3,activation='relu'))
model.add(MaxPool2D(pool_size=2 , strides=2))

In [74]:
model.add(Conv2D(filters=512,kernel_size=3,padding='same',activation='relu'))
model.add(Conv2D(filters=512,kernel_size=3,activation='relu'))
model.add(MaxPool2D(pool_size=2 , strides=2))

In [75]:
model.add(Dropout(0.25))   #to avoid overfitting

In [76]:
model.add(Flatten())

In [77]:
model.add(Dense(units=1500,activation='relu')) ##Increase neurons

In [78]:
model.add(Dropout(0.4))

In [79]:
##output layer
model.add(Dense(units=38, activation='softmax'))

## Compiling Model

### to avoid overshooting
1. we repalace learning rate from default 0.001 to 0.0001
2. there may be chance of underfitting , so increase number of neuron
3. we add more convuation layer to extract more feature from image there may be possibility that model is confusing due to lack of feature

In [80]:
model.compile(optimizer=tf.keras.optimizers.Adam(
    learning_rate=0.0001),loss='categorical_crossentropy', metrics=['accuracy'])

In [81]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_19 (Conv2D)          (None, 128, 128, 32)      896       
                                                                 
 conv2d_20 (Conv2D)          (None, 126, 126, 32)      9248      
                                                                 
 max_pooling2d_7 (MaxPooling  (None, 63, 63, 32)       0         
 2D)                                                             
                                                                 
 conv2d_21 (Conv2D)          (None, 63, 63, 64)        18496     
                                                                 
 conv2d_22 (Conv2D)          (None, 61, 61, 64)        36928     
                                                                 
 max_pooling2d_8 (MaxPooling  (None, 30, 30, 64)       0         
 2D)                                                  

## Model training ---

In [82]:
training_history=model.fit(x=training_set,validation_data=validation_set,epochs=10)


Epoch 1/10
550/550 [==============================] - 1295s 2s/step - loss: 2.5483 - accuracy: 0.2891 - val_loss: 1.4431 - val_accuracy: 0.5760
Epoch 2/10
550/550 [==============================] - 1300s 2s/step - loss: 1.2689 - accuracy: 0.6173 - val_loss: 0.7614 - val_accuracy: 0.7686
Epoch 3/10
550/550 [==============================] - 1315s 2s/step - loss: 0.8398 - accuracy: 0.7420 - val_loss: 0.6491 - val_accuracy: 0.7988
Epoch 4/10
550/550 [==============================] - 1285s 2s/step - loss: 0.6191 - accuracy: 0.8037 - val_loss: 0.3864 - val_accuracy: 0.8758
Epoch 5/10
550/550 [==============================] - 1299s 2s/step - loss: 0.4661 - accuracy: 0.8513 - val_loss: 0.3555 - val_accuracy: 0.8834
Epoch 6/10
550/550 [==============================] - 1362s 2s/step - loss: 0.3508 - accuracy: 0.8847 - val_loss: 0.2049 - val_accuracy: 0.9345
Epoch 7/10
550/550 [==============================] - 1427s 3s/step - loss: 0.2997 - accuracy: 0.9013 - val_loss: 0.1681 - val_accuracy: